## Portfolio Tools and Optimization in Python

##### Author: Zachary Wright, CFA, FRM | Last Updated: 02/02/25

Welcome! This is a portfolio project dedicated toward showcasing skills acquired in Python in a quantitative finance scenario and may be updated as of time of viewing.

**Goal and problem definition:** optimize a portfolio using various methods such as maximize Sharpe ratio and minimize variance.

Tools used:
- yfinance API to gather asset return data dynamically
- Scipy library to optimize asset weightings
- Python class constructor and object-oriented programming techniques for efficiency and modularity

First I will install and then import the required libraries:

In [1]:
!pip install pandas
!pip install numpy
!pip install yfinance
!pip install scipy


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

Next, since Python is an interpreted language, I will define a Portfolio class and its associated functions and methods first.

In terms of a portfolio class, we can image multiple different portfolios that we want to get return and variance data for.

Methods defined:
- Get yahoo finance asset returns
- Get portfolio return (default is monthly)
- Get portfolio covariance
- Get portfolio variance (default is monthly)
- Optimize (maximize Sharpe ratio, minimize variance), default is maximize Sharpe

In [3]:
#Create class portfolio - we can theoretically have multiple portfolios with different asset and return data series.
class Portfolio:
    #Initialize instance of portfolio class w/ parameters returns and weights
    def __init__(self, returns, weights):
        self.returns = returns
        self.weights = weights

    #Get return data from respective tickers - default interval is 1 month data.
    def get_returns_from_yfinance(self, tickers, start_date, end_date, interval="1mo"):
        all_data = {}
        for ticker in tickers:
            try:
                print(f"Fetching data for {ticker}...\n")
                data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
                
                #Handle multi-index columns
                if isinstance(data.columns, pd.MultiIndex):
                    #Access the specific 'Close' column for this ticker
                    if ('Close', ticker) in data.columns:
                        prices = data[('Close', ticker)]
                    else:
                        print(f"No valid 'Close' column found for {ticker}.")
                        continue
                else:
                    print(f"Unexpected column structure for {ticker}.")
                    continue
                
                #Calculate percentage returns
                returns = prices.pct_change().dropna()
                all_data[ticker] = returns
            except Exception as e:
                print(f"Error fetching data for {ticker}: {e}")
        
        if all_data:
            return pd.DataFrame(all_data)
        else:
            raise ValueError("No valid data fetched. Please check the tickers or date range.")

    #Return the return of portfolio
    def get_return(self, weights=None):
        avg_asset_returns = self.returns.mean()
        if weights is None:
            weights = self.weights
        port_return = sum(w * avg_ret for w, avg_ret in zip(weights, avg_asset_returns))
        return port_return

    #Return the covariance of the portfolio
    def get_cov(self):
        return self.returns.cov()

    #Return the variance of the portfolio
    def get_var(self, weights=None):
        if weights is None:
            weights = self.weights
        cov_matrix = self.get_cov()
        weights = np.array(weights)
        port_variance = weights.T @ cov_matrix @ weights
        return port_variance

    #Optimization function: can maximize Sharpe Ratio, minimize variance, and target return and risk
    def optimize(self, objective="sharpe", target_return=None, target_risk=None):
        n_assets = len(self.weights)
        avg_returns = self.returns.mean()
        cov_matrix = self.get_cov()

        #Risk-free rate proxy from TBLL ETF yield through yfinance
        sgov = yf.Ticker("SGOV")
        rfr_proxy = sgov.info.get('dividendYield', 4.3)
        print("Risk Free Rate Assumption (3-Mo. Treasury Yield: ", "{:.2f}%".format(rfr_proxy), "\n")
        
        #Define constraints
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(n_assets)]

        #Define the objective function
        if objective == "sharpe":
            def objective_function(weights):
                port_return = self.get_return(weights)
                port_variance = self.get_var(weights)
                sharpe_ratio = (port_return - rfr_proxy) / np.sqrt(port_variance)
                return -sharpe_ratio #Negative to maximize sharpe
        elif objective == "min_variance":
            def objective_function(weights):
                return self.get_var(weights)
        else:
            raise ValueError("Invalid optimization objective.")

        #Optimize
        result = minimize(objective_function, self.weights, constraints=constraints, bounds=bounds)
        if result.success:
            self.weights = result.x
        return result

From here, we can import our portfolio's assets and their associated monthly returns. 

In [4]:
#Define tickers and date range
tickers = ["IVV", "IXUS", "IEMG"]
weights = [0.65,0.20,0.15]
start_date = "2020-01-01"
end_date = "2023-12-31"
opt_objective = "sharpe"

#Initialize dummy Portfolio object with placeholders
dummy_returns = pd.DataFrame()  #Placeholder returns (will be replaced)
dummy_weights = [1 / len(tickers)] * len(tickers)  #Equal weighting
portfolio = Portfolio(dummy_returns, dummy_weights)

#Fetch returns from Yahoo Finance
try:
    portfolio_returns = portfolio.get_returns_from_yfinance(tickers, start_date, end_date)
    print("Fetched Returns, displaying first five rows: ")
    print(portfolio_returns.head(),"\n")
except ValueError as e:
    print(e)
    portfolio_returns = None

#Create a new Portfolio object with fetched returns
if portfolio_returns is not None:
    #weights = [1 / len(tickers)] * len(tickers)
    portfolio = Portfolio(portfolio_returns, weights)

    #Initial Portfolio Stats
    initial_return = portfolio.get_return() * 100
    initial_variance = portfolio.get_var()  
    print("Old Weights: \n")
    for ticker, weight in zip(tickers, weights):
        print(f"  {ticker}: {weight * 100:.2f}%\n")
    print(f"Initial MonthlyPortfolio Return: {initial_return:.2f}%")
    print(f"Initial Monthly Portfolio Variance: {initial_variance:.6f}\n")

    #Optimization Output
    optimized_result = portfolio.optimize(objective=opt_objective)
    
    print(f"Optimized Weights via {opt_objective}: ")
    for ticker, weight in zip(tickers, optimized_result.x):
        print(f"  {ticker}: {weight * 100:.2f}%\n")
    
    optimized_return = portfolio.get_return(optimized_result.x) * 100
    optimized_variance = portfolio.get_var(optimized_result.x)  #Variance in raw terms
    print(f"Optimized Monthly Portfolio Return: {optimized_return:.2f}%")
    print(f"Optimized Monthly Portfolio Variance: {optimized_variance:.6f}")

Fetching data for IVV...



[*********************100%***********************]  1 of 1 completed

*********************100%***********************]  1 of 1 completed

Fetching data for IXUS...

Fetching data for IEMG...




*********************100%***********************]  1 of 1 completed

Fetched Returns, displaying first five rows: 
                 IVV      IXUS      IEMG
Date                                    
2020-02-01 -0.084550 -0.069915 -0.037161
2020-03-01 -0.126762 -0.156620 -0.169165
2020-04-01  0.133840  0.071049  0.078824
2020-05-01  0.048152  0.048659  0.031608
2020-06-01  0.014779  0.032576  0.056838 

Old Weights: 

  IVV: 65.00%

  IXUS: 20.00%

  IEMG: 15.00%

Initial MonthlyPortfolio Return: 0.89%
Initial Monthly Portfolio Variance: 0.003100

Risk Free Rate Assumption (3-Mo. Treasury Yield:  4.30% 

Optimized Weights via sharpe: 
  IVV: 100.00%

  IXUS: 0.00%

  IEMG: 0.00%

Optimized Monthly Portfolio Return: 1.13%
Optimized Monthly Portfolio Variance: 0.003447


Thank you for viewing this portfolio project!